# Zugriffsdaten-Analyse

## Datenimport

### Parsing

In [2]:
import apachelogs

parser = apachelogs.LogParser('{} "%{{Unknown}}i"'.format(apachelogs.COMBINED))

#### Beispiel

In [6]:
data = parser.parse('207.46.13.115 - - [22/Jan/2019:03:56:29 +0330] "GET /image/46131/productModel/100x100 HTTP/1.1" 200 1981 "-" "Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)" "-"\n').directives

print("Host: {}".format(data["%h"]))
print("Logname: {}".format(data["%l"]))
print("User: {}".format(data["%u"]))
print("Timestamp: {}".format(data["%t"]))
print("Request line: {}".format(data["%r"]))
print("Status: {}".format(data["%>s"]))
print("Bytes sent: {}".format(data["%b"]))
print("Referer: {}".format(data["%{Referer}i"]))
print("User-Agent: {}".format(data["%{User-Agent}i"]))

Host: 207.46.13.115
Logname: None
User: None
Timestamp: 2019-01-22 03:56:29+03:30
Request line: GET /image/46131/productModel/100x100 HTTP/1.1
Status: 200
Bytes sent: 1981
Referer: None
User-Agent: Mozilla/5.0 (compatible; bingbot/2.0; +http://www.bing.com/bingbot.htm)


### Einlesen der Datei

In [3]:
from pyspark.sql import Row
import pandas as pd

file = open('data/access.log')
rows = []
for line in file.readlines():
    entry = parser.parse(line).directives
    rows.append({
        "host": entry["%h"],
        "logname": entry["%l"],
        "user": entry["%u"],
        "timestamp": entry["%t"],
        "request_line": entry["%r"],
        "status": entry["%>s"],
        "bytes_sent": entry["%b"],
        "referer": entry["%{Referer}i"],
        "user_agent": entry["%{User-Agent}i"],
    })
file.close()

pd_df = pd.DataFrame(rows, columns=rows[0].keys())
pd_df.head()

,host,logname,user,timestamp,request_line,status,bytes_sent,referer,user_agent
0,54.36.149.41,None,None,2019-01-22 03:56:14+03:30,GET /filter/27|13%20%D9%85%DA%AF%D8%A7%D9%BE%D...,200,30577,None,Mozilla/5.0 (compatible; AhrefsBot/6.1; +http:...
1,31.56.96.51,None,None,2019-01-22 03:56:16+03:30,GET /image/60844/productModel/200x200 HTTP/1.1,200,5667,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
2,31.56.96.51,None,None,2019-01-22 03:56:16+03:30,GET /image/61474/productModel/200x200 HTTP/1.1,200,5379,https://www.zanbil.ir/m/filter/b113,Mozilla/5.0 (Linux; Android 6.0; ALE-L21 Build...
3,40.77.167.129,None,None,2019-01-22 03:56:17+03:30,GET /image/14925/productModel/100x100 HTTP/1.1,200,1696,None,Mozilla/5.0 (compatible; bingbot/2.0; +http://...
4,91.99.72.15,None,None,2019-01-22 03:56:17+03:30,GET /product/31893/62100/%D8%B3%D8%B4%D9%88%D8...,200,41483,None,Mozilla/5.0 (Windows NT 6.2; Win64; x64; rv:16...


In [4]:
import os
os.makedirs("/tmp/data", exist_ok=True)
pd_df.to_csv("/tmp/data/access.log.csv", index=False)

## Spark

### Setup

In [1]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName("main").master("spark://localhost:7077").config("spark.executor.memory", "5G").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 13:34:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("/tmp/data/access.log.csv", header=True)

In [3]:
df.count()

10365152

In [4]:
df.show()

+-------------+-------+----+--------------------+--------------------+------+----------+--------------------+--------------------+
|         host|logname|user|           timestamp|        request_line|status|bytes_sent|             referer|          user_agent|
+-------------+-------+----+--------------------+--------------------+------+----------+--------------------+--------------------+
| 54.36.149.41|   null|null|2019-01-22 03:56:...|GET /filter/27|13...|   200|     30577|                null|Mozilla/5.0 (comp...|
|  31.56.96.51|   null|null|2019-01-22 03:56:...|GET /image/60844/...|   200|      5667|https://www.zanbi...|Mozilla/5.0 (Linu...|
|  31.56.96.51|   null|null|2019-01-22 03:56:...|GET /image/61474/...|   200|      5379|https://www.zanbi...|Mozilla/5.0 (Linu...|
|40.77.167.129|   null|null|2019-01-22 03:56:...|GET /image/14925/...|   200|      1696|                null|Mozilla/5.0 (comp...|
|  91.99.72.15|   null|null|2019-01-22 03:56:...|GET /product/3189...|   200|     4

In [15]:
df.createOrReplaceTempView("access_log")

### SQL

In [23]:
query = spark.sql("SELECT host, status, user_agent FROM access_log WHERE status = 400")

In [24]:
query.count()

585

In [26]:
query.show()

+---------------+------+--------------------+
|           host|status|          user_agent|
+---------------+------+--------------------+
|  61.219.11.151|   400|                null|
| 178.47.232.191|   400|           Yowai/2.0|
| 37.255.249.243|   400|Mozilla/5.0 (comp...|
|   151.25.29.64|   400|        Solstice/2.0|
| 188.19.139.184|   400|           Yowai/2.0|
|  42.112.163.67|   400|           Yowai/2.0|
|   80.82.64.127|   400|                null|
|   80.82.64.127|   400|    libwww-perl/6.36|
|  164.52.24.162|   400|Mozilla/5.0 (Wind...|
|   222.95.88.49|   400|                null|
|   222.95.88.49|   400|                null|
| 126.60.209.224|   400|         Tsunami/2.0|
|   72.52.125.78|   400|                null|
| 37.255.249.243|   400|Mozilla/5.0 (comp...|
|220.181.124.101|   400|Sogou web spider/...|
| 106.38.241.157|   400|Sogou web spider/...|
|  119.93.85.191|   400|         Cayosin/2.0|
|   93.115.27.22|   400|                null|
|   93.115.27.22|   400|          